In [9]:
!pip3 install lightgbm mljar-supervised scikit-learn tqdm catboost optuna xgboost ipywidgets autogluon
# 모델을 위해 필요한 라이브러리 설치(우수 예시에만 있는 부분)

In [10]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import random
import os
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

import warnings
warnings.filterwarnings('ignore')


In [11]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed=42
seed_everything(seed) # Seed 고정

In [12]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [13]:
def category_all(df_train, df_test):
    label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    ]
    for i in label_columns:
        df_train[i] = df_train[i].astype('category')
        df_test[i] = df_test[i].astype('category')
    return df_train, df_test    


def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""
    
    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

        
def manage_label_encoding(df_train, df_test):
    tmp = len(df_train)
    label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    ]
    
    df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

    for col in label_columns:
        df_all[col] = label_encoding(df_all[col])
    
    for col in label_columns:  
        df_train[col] = df_all.iloc[: tmp][col]
        df_test[col] = df_all.iloc[tmp :][col]

    return df_train, df_test

def preprocessing(df_train, df_test):
    df_train.drop_duplicates(keep='first', inplace=True)
    df_train.reset_index(inplace=True)
    df_train.drop('index',inplace=True, axis=1)
    df_test_id_temp = df_test['id'].copy()
    df_all = pd.concat([df_train,df_test.drop('id',axis=1)])
    df_all.drop('customer_country.1',axis=1,inplace=True)
    df_all.id_strategic_ver.fillna(0., inplace=True)
    df_all.it_strategic_ver.fillna(0., inplace=True)
    df_all.idit_strategic_ver.fillna(0., inplace=True)
    df_all.business_subarea.fillna('NAN', inplace=True)
    df_all.com_reg_ver_win_rate.fillna(0,inplace=True)
    df_all['customer_type'] = df_all['customer_type'].replace({
        'End-Customer':'End Customer'
    })
    df_all.customer_type.fillna('NAN',inplace=True)
    df_all.historical_existing_cnt.fillna(0, inplace=True)
    
    df_all.product_subcategory.fillna('NAN', inplace=True)
    
    df_all.product_modelname.fillna('NAN', inplace=True)

    df_all['expected_timeline'] = df_all['expected_timeline'].replace({
        'less_than_3_months': 'less than 3 months',  # merge into 'less than 3 months'
        'being followed up': 'being followed up.',  # merge into 'being followed up.'
        'less than 6 months': '3 months ~ 6 months',  # merge into '3 months ~ 6 months'
        '3_months_~_6_months' : '3 months ~ 6 months'
    })

    df_all.expected_timeline.fillna('NAN',inplace=True)
    df_all.ver_win_rate_x.fillna(0,inplace=True)
    df_all.ver_win_ratio_per_bu.fillna(0,inplace=True)
    df_all.business_area.fillna('NAN',inplace=True)
    df_all.customer_job.fillna('NAN', inplace=True)
    df_all.product_category.fillna('NAN', inplace=True)
    
    #처리못한부분
    #customer_country
    df_all.customer_country.fillna('NAN', inplace=True)
    #inquiry_type
    df_all.inquiry_type.fillna('NAN', inplace=True)

    label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    ]
    
    for i in label_columns:
        df_all[i] = df_all[i].str.lower()
        df_all[i] = df_all[i].str.replace(pat=r'[^\w]', repl=r'', regex=True)
        
    tmp_len = len(df_train)
    df_train = df_all.iloc[: tmp_len ]
    df_test = df_all.iloc[tmp_len :]
    df_test = pd.concat([df_test,df_test_id_temp],axis=1)
    return df_train, df_test
df_train, df_test = preprocessing(df_train, df_test)

In [14]:
train_data = TabularDataset(df_train)
test_data = df_test.drop(["is_converted", "id"], axis=1)
test_data = TabularDataset(test_data)

In [16]:
fit_parameters = {
    'presets': 'best_quality',
    'num_bag_folds': 5,  ### 값을 키울수록 점수가 좋아지나, 너무 키우면 검증세트 크기가 작아져 잘못된 결과를 얻게 됨
    'num_bag_sets': 3,  ### 값을 키울수록 점수가 좋아짐, 속도가 느려짐.
    'num_stack_levels': 0,  ### 2번이 의미가 없어, 1번만 해도 됨
    'excluded_model_types': ['NN_TORCH'],  ### 속도가 느린 모델은 제외시켰습니다.
}

predictor = TabularPredictor(
    label='is_converted',
    problem_type='binary',
    eval_metric='f1',
    sample_weight='auto_weight',
    log_file_path=False
).fit(train_data, 
      **fit_parameters,
      time_limit=3600*2,
      #num_gpus=1,
     )

No path specified. Models will be saved in: "AutogluonModels/ag-20240208_213603"
Presets specified: ['best_quality']
Dynamic stacking was enabled but stacking is disabled. Setting dynamic stacking to False.
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=5, num_bag_sets=3
Beginning AutoGluon training ... Time limit = 7200s
AutoGluon will save models to "AutogluonModels/ag-20240208_213603"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.11.5
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #102~20.04.1-Ubuntu SMP Mon Jan 15 13:09:14 UTC 2024
CPU Count:          16
Memory Avail:       114.78 GB / 125.66 GB (91.3%)
Disk Space Avail:   1596.05 GB / 1832.21 GB (87.1%)
Train Data Rows:    55780
Train Data Columns: 27
Label Column:       is_converted
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = True, class 0 = False
Using Feature Generators to

KeyboardInterrupt: 

In [46]:
print(predictor.leaderboard(silent = True)) 

                       model  score_val eval_metric  pred_time_val   fit_time  \
0        WeightedEnsemble_L2   1.622370   f1_recall       1.493873  41.735932   
1    RandomForest_r16_BAG_L1   1.620407   f1_recall       0.713848   6.531717   
2   RandomForest_r195_BAG_L1   1.616083   f1_recall       0.714703   4.750946   
3    RandomForestGini_BAG_L1   1.572053   f1_recall       0.761032   1.537467   
4    RandomForestEntr_BAG_L1   1.569848   f1_recall       0.740138   1.421936   
5     ExtraTrees_r197_BAG_L1   1.567019   f1_recall       0.773014   1.974630   
6    RandomForest_r39_BAG_L1   1.565111   f1_recall       0.716214   4.640099   
7   RandomForest_r166_BAG_L1   1.564803   f1_recall       0.765261   1.347649   
8      ExtraTrees_r42_BAG_L1   1.559119   f1_recall       0.790327   1.557582   
9    RandomForest_r15_BAG_L1   1.524882   f1_recall       0.712489   4.608275   
10  RandomForest_r127_BAG_L1   1.491273   f1_recall       0.703830   6.123113   
11     KNeighborsDist_BAG_L1

In [54]:
test_pred = predictor.predict(TabularDataset(df_train.drop(["is_converted"], axis=1).fillna(0)), model=predictor.get_model_best())
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))
    
get_clf_eval(df_train['is_converted'],test_pred)

오차행렬:
 [[ 4835    15]
 [    5 54444]]

정확도: 0.9997
정밀도: 0.9990
재현율: 0.9969
F1: 0.9979


In [52]:
test_pred = predictor.predict_proba(df_train.drop(["is_converted"], axis=1).fillna(0), model=predictor.get_model_best())
y = df_train['is_converted']
threshold = min([num[1] if y[i] else 1 for i, num in enumerate(np.array(test_pred))])
threshold

0.32545867562294006

In [60]:
test_pred = predictor.predict_proba(test_data, model=predictor.get_model_best())
pred = [True if i[1]>=threshold else False for i in np.array(test_pred)]

sum(pred)

1496

In [61]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**